In [ ]:
"""This file gets technicals for a given ticker, with the addition of fundamentals"""

In [ ]:
!pip3 install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=140b16b84c2f1a5fa792249e85f694a5120e1f2b45b62d7bad06eaa84ad2cfb4
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
import requests
import pandas as pd
import ta

In [ ]:
API_KEY = 'szGfkmz_loFe5Gt3FfJQzCGu1jlKa_uL'
BASE_URL = 'https://api.polygon.io'

In [ ]:
def get_historical_data(ticker, start_date, end_date, timespan='day'):
    url = f"{BASE_URL}/v2/aggs/ticker/{ticker}/range/1/{timespan}/{start_date}/{end_date}"
    params = {'apiKey': API_KEY}
    response = requests.get(url, params=params)
    data = response.json()

    if 'results' not in data:
        raise ValueError(f"Error fetching data: {data.get('error', 'Unknown error')}")

    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.rename(columns={'c': 'close', 'o': 'open', 'h': 'high', 'l': 'low', 'v': 'volume'})
    return df[['open', 'high', 'low', 'close', 'volume']]

In [ ]:
#You can change the parameters to whatever best fits the algorithm
#Maybe have 2 functions, one with longer periods for longer term strategies -- Aryan
def calculate_indicators(df, sma_period=50, ema_period=50, macd_fast=12, macd_slow=26, macd_signal=9, rsi_period=14):
    """
    Add technical indicators (SMA, EMA, MACD, RSI) to the DataFrame.
    """
    # Calculate Simple Moving Average (SMA)
    df[f'SMA_{sma_period}'] = ta.trend.sma_indicator(df['close'], window=sma_period)

    # Calculate Exponential Moving Average (EMA)
    df[f'EMA_{ema_period}'] = ta.trend.ema_indicator(df['close'], window=ema_period)

    # Calculate MACD
    df['MACD'] = ta.trend.macd(df['close'], window_slow=macd_slow, window_fast=macd_fast)
    df['MACD_Signal'] = ta.trend.macd_signal(df['close'], window_slow=macd_slow, window_fast=macd_fast, window_sign=macd_signal)
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']

    # Calculate Relative Strength Index (RSI)
    df['RSI'] = ta.momentum.rsi(df['close'], window=rsi_period)

    return df

In [ ]:
ticker = "AAPL"
start_date = "2023-01-01"
end_date = "2024-12-31"

data = get_historical_data(ticker, start_date, end_date)
data_with_indicators = calculate_indicators(data)
print(data_with_indicators.tail())

                       open    high     low   close      volume    SMA_50  \
timestamp                                                                   
2024-12-24 05:00:00  255.49  258.21  255.29  258.20  22420512.0  235.9140   
2024-12-26 05:00:00  258.19  260.10  257.63  259.02  26226173.0  236.4174   
2024-12-27 05:00:00  257.83  258.70  253.06  255.59  40542367.0  236.8936   
2024-12-30 05:00:00  252.23  253.50  250.75  252.20  33166048.0  237.2946   
2024-12-31 05:00:00  252.44  253.28  249.43  250.42  34284191.0  237.6030   

                         EMA_50      MACD  MACD_Signal  MACD_Histogram  \
timestamp                                                                
2024-12-24 05:00:00  238.380336  6.053677     5.368402        0.685274   
2024-12-26 05:00:00  239.189735  6.288037     5.552329        0.735708   
2024-12-27 05:00:00  239.832883  6.126376     5.667138        0.459237   
2024-12-30 05:00:00  240.317868  5.659475     5.665606       -0.006131   
2024-12-31 05:00

In [ ]:
ticker = 'NVDA'
url = f'https://api.polygon.io/v2/reference/financials/{ticker}?apiKey={API_KEY}'

response = requests.get(url)
data = response.json()

# Check if the response contains financial data
if 'results' in data:
    financial_data = data['results'][0]
    eps = financial_data.get('earningsPerBasicShare', 'Not available')
    fcf = financial_data.get('freeCashFlow', 'Not available')
    pe_ratio = financial_data.get('priceToEarningsRatio', 'Not available')
    roe = financial_data.get('returnOnEquity', 'Not available')
    de_ratio = financial_data.get('debtToEquityRatio', 'Not available')

    netIncome = financial_data.get('netIncome', 'Not available')
    shareholdersEquity = financial_data.get('shareholdersEquity', 'Not available')
    roe = netIncome / shareholdersEquity


    #print(f'TTM EPS: {ttm_eps}')
    print(f'EPS: {eps}')
    print(f'Free Cash Flow: {fcf}')
    print(f'P/E Ratio: {pe_ratio}')
    print(f'ROE: {roe}')
    print(f'D/E Ratio: {de_ratio}')
    print("\n")
else:
    print("No financial data available.")


EPS: 1.56
Free Cash Flow: 1320000000
P/E Ratio: 67.255
ROE: 0.07792527040314651
D/E Ratio: 0.419




In [ ]:
data['results']

[{'ticker': 'NVDA',
  'period': 'Q',
  'calendarDate': '2019-12-31',
  'reportPeriod': '2020-01-26',
  'updated': '2020-02-21',
  'dateKey': '2020-02-20',
  'accumulatedOtherComprehensiveIncome': 1000000,
  'assets': 17315000000,
  'assetsCurrent': 13690000000,
  'assetsNonCurrent': 3625000000,
  'bookValuePerShare': 20.039,
  'capitalExpenditure': -145000000,
  'cashAndEquivalents': 10896000000,
  'cashAndEquivalentsUSD': 10896000000,
  'costOfRevenue': 1090000000,
  'consolidatedIncome': 951000000,
  'currentRatio': 7.674,
  'debtToEquityRatio': 0.419,
  'debt': 2552000000,
  'debtCurrent': 0,
  'debtNonCurrent': 2552000000,
  'debtUSD': 2552000000,
  'deferredRevenue': 0,
  'depreciationAmortizationAndAccretion': 106000000,
  'deposits': 0,
  'dividendYield': 0.002,
  'dividendsPerBasicCommonShare': 0.16,
  'earningBeforeInterestTaxes': 1029000000,
  'earningsBeforeInterestTaxesDepreciationAmortization': 1135000000,
  'EBITDAMargin': 0.366,
  'earningsBeforeInterestTaxesDepreciation